# Advanced stitching
## Stitcher tuning

In this tutorial you will learn how to stitch multichannel data and how to change the stitcher option to obtain the best stitching results.

In [ ]:
import paprica
import matplotlib.pyplot as plt

path = '../../../tests/data/apr'
tiles = paprica.tileParser(path=path, ftype='apr', frame_size=512)
stitcher = paprica.tileStitcher(tiles, overlap_h=25, overlap_v=25)

By default, the stitcher uses entire tiles to compute the registration between each tile. If the tiles contains a lot of information or particular structures, it
might be necessary to reduce the number of planes included in the computation. To do so we provide a ´set_z_range´ method that here restrict the computation between plane 100 and 200:

In [ ]:
stitcher.set_z_range(z_begin=100, z_end=200)

The overlap margin between tile can also be modified. By default, the pipeline use 20% more than the given overlap to register the tiles. Ideally this margin should be small enough to avoid spurious matches but **must** be large enough that the true overlaps are actually smaller than the one used for registration. If the true overlap is larger than the one used for the computation, then it is impossible for the pipeline to register correctly the tiles. Note that this limitation exists for many other pipelines and is not APR specific. The following allows to set the margin to 5%:

In [ ]:
stitcher.set_overlap_margin(5)

We also provide a way to avoid large displacements that can happen e.g. between two empty tiles. It basically sets the registration output to the expected one (given on the stitcher instantiation) if the computed displacement differs more than the one that is set. The associated reliability will be set low so that the global optimisation should avoid this computation anyway. In the following we set the regularisation parameter to 20 pixels for each dimension:

In [ ]:
stitcher.set_regularization(reg_x=20, reg_y=20, reg_z=20)

## Multichannel stitching

The way multiple channel are stitched is very easy: we start by stitching a master channel (ideally the one that has nice signal and SNR) and we then register the other channels on the master channel. Let's start by computing the registration of the previously loaded master channel:

In [ ]:
stitcher.compute_registration(progress_bar=False)

Then we can parse a second channel and register it to the master one:

In [ ]:
tiles_ch2 = paprica.tileParser(path='path to second channel', ftype='apr', frame_size=512)
stitcher = paprica.channelStitcher(stitcher=stitcher, ref=tiles, moving=tiles_ch2)